In [1]:
from Sim import *


import collections

import pdb
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import time
from tqdm import tqdm

reward: -0.1 -0.1 10


/home/zlxlekta924/anaconda3/envs/test/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



# DQN Agent

In [2]:
'''
Architecture of Layers

Dqn 에서의 구조
84x84x4 사이즈
순서대로 input, channel, kernel, strdides
Conv2d(x, 32, 8, 4)
conv2d(x, 64, 4, 2)
conv2d(x, 64, 3, 1)
flatten()
dense(x, 512)
dense(x, 2)

이 때 우리는
9x10x1 이고 약 300배 차이가 나므로

Conv2d(x, 16, 4, 2)
conv2d(x, 32, 3, 1)
flatten()
dense(x, 128)
dense(x, 4)

로 변경하면 괜찮을 듯
'''

class QnetCNN(nn.Module):
    def __init__(self):
        super(QnetCNN, self).__init__()
        
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=4, stride=2),
            nn.ReLU())

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1),
            nn.ReLU())
    
        #Flatten
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 , 32)
        self.fc2 = nn.Linear(32, 4)
        
    def forward(self,x):
        
        x = torch.reshape(x, (-1, 1, 10, 9)).to(device)

        #pdb.set_trace()
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)# 4개의 값을 반환
        return x

    def sample_action(self, obs, epsilon):
        out = self.forward(obs) 
        coin = random.random() 
        

        if coin < epsilon:  # 동전던지기, eps 확률로 랜덤하게 행동
            return random.randint(0,3)
        else:
            return out.argmax().item() #Action state 값이 더 높은 index 값 호출


def train(q, q_target, memory, optimizer): 
    for i in range(10):
        s, a, r, s_prime, done_mask = memory.sample(batch_size) #32개를 버퍼에서 뽑아 모아 놓은 s,a,r,s_prime,done_mask

        q_out = q(s)                                   # s 값으로 다음 각 action 값들의 value 값 반환
        q_a = q_out.gather(1,a.to(device))         #선택한 액션값들의 q(s,a) 반환
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1) # 다음 state의 각 q(s,a) 값 반환
        target = torch.tensor(r, device=device) + torch.tensor(gamma, device=device) * max_q_prime * torch.tensor(done_mask, device=device) # 배열 맞춰주기, 쓰러진 경우는 제거
        loss = F.smooth_l1_loss(q_a, target) # DQN 의 손실함수 계산 L1 유클리드

        optimizer.zero_grad() # optimizer 의 모든 parameter 를 0으로 변환
        loss.backward() # loss 에 대한 gradient 계산
        optimizer.step() # 손실값을 바탕으로 Qnet 의 파라미터 업데이트

def model_save(model_dict, opt_dict, epi):
    PATH = './weights/'
    torch.save({
            'model': model_dict,
            'optimizer': opt_dict,
            'epi': epi
            }, PATH + 'all.tar')

## Replay Buffer

In [3]:
class ReplayBuffer():
    
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)

    def put(self, transition):
        self.buffer.append(transition)

    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []

        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch. float), torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
        
    def size(self):
        return len(self.buffer)


## Step

In [8]:
import collections
from Sim import *


learning_rate = 0.0005
gamma = 0.99
buffer_limit  = 50000
batch_size = 32

USE_CUDA = torch.cuda.is_available()
device = torch.device('cuda:0' if USE_CUDA else 'cpu')

env = Simulator()

memory = ReplayBuffer()
q = QnetCNN().to(device)
q_target = QnetCNN().to(device)
q_target.load_state_dict(q.state_dict()) # 현재 Qnet 의 파라미터를 q_target 에 load

optimizer = optim.Adam(q.parameters(), lr = learning_rate) # loss 값을 바탕으로 업데이트할 비율 (q_target 말고 q 만 업데이트)

print_interval = 50
score = 0.0
files = pd.read_csv("./data/factory_order_train.csv")
    
for epi in tqdm(range(40000)): 

    if epi%100 == 0:
        model_save(q.state_dict(), optimizer.state_dict(), epi)
        
    items = list(files.iloc[0])[0]
    
    epsilon = max(0.01, 0.50 - 0.02 * (epi//100))

    s = env.reset(0)
    obs = np.asarray(s, dtype=np.float32)
    
    done = False
    first = True
    a_step = 0
    
    while not done:
        
        # Action choose
        if first:
            a = 0
            first = False
        
        else:
            a = q.sample_action(torch.from_numpy(obs).float(), epsilon) 
        s_prime, r, cumul, done, goal_reward = env.step(a)
        
        array = s_prime
        action = ['↑', '↓', '←', '→'][a]
        s_prime = np.asarray(s_prime, dtype=np.float32)
        
        # Calculate
        done_mask = 0.0 if done else 1.0
        memory.put((obs,a,r, s_prime, done_mask))
        
        score += r
        a_step +=1
        
        if a_step ==100: #1000번 했는데도 안되면 강제종료
            done =True
            goal_reward=True
        
        if done :
            break
            
    if memory.size() > 4000:
        train(q, q_target, memory, optimizer)
        
    if epi%print_interval==0 and epi != 0: 
        print(f'{epi} 에서 챙기는 아이템 : {items}\n마지막 행동 : {action}\n 그 때의 마지막 상태 :\n{array}\n마지막 보상 : {r}\n 총 받은 보상 :{cumul}')
        q_target.load_state_dict(q.state_dict()) #q_target 지금걸로 업데이트
        print(f"score = {int(score/print_interval)}, n_buffer : {memory.size()}, eps : {epsilon*100}")
        score = 0.0



  0%|                                                                     | 51/40000 [00:05<1:28:03,  7.56it/s]

50 에서 챙기는 아이템 : ['H', 'L', 'M']
마지막 행동 : ↓
 그 때의 마지막 상태 :
[[0.  0.  0.  0.  0.  0.  0.  0.6 0. ]
 [1.  1.  1.  1.  1.  1.  1.  1.  1. ]
 [0.  1.  1.  1.  0.3 1.  1.  1.  0. ]
 [0.  1.  0.  1.  0.  1.  0.  1.  0. ]
 [0.  1.  0.  1.  0.  1.  0.  1.  0. ]
 [0.  1.  0.  1.  0.  1.  0.  1.  0. ]
 [1.  1.  0.  1.  0.  1.  0.  1.  1. ]
 [1.  1.  1.  1.  1.  1.  1.  1.  1. ]
 [1.  1.  1.  1.  1.  1.  1.  1.  1. ]
 [0.  0.  0.  0.  1.  0.  0.  0.  0. ]]
마지막 보상 : -0.1
 총 받은 보상 :0.1000000000000068
score = -9, n_buffer : 5100, eps : 50.0


  0%|▏                                                                   | 101/40000 [00:15<2:17:02,  4.85it/s]

100 에서 챙기는 아이템 : ['H', 'L', 'M']
마지막 행동 : ↑
 그 때의 마지막 상태 :
[[0.  0.  0.  0.  0.  0.  0.  0.6 0. ]
 [1.  1.  1.  0.3 1.  1.  1.  1.  1. ]
 [0.  1.  1.  1.  1.  1.  1.  1.  0. ]
 [0.  1.  0.  1.  0.  1.  0.  1.  0. ]
 [0.  1.  0.  1.  0.  1.  0.  1.  0. ]
 [0.  1.  0.  1.  0.  1.  0.  1.  0. ]
 [1.  1.  0.  1.  0.  1.  0.  1.  1. ]
 [1.  1.  1.  1.  1.  1.  1.  1.  1. ]
 [1.  1.  1.  1.  1.  1.  1.  1.  1. ]
 [0.  0.  0.  0.  1.  0.  0.  0.  0. ]]
마지막 보상 : -0.1
 총 받은 보상 :0.10000000000001169
score = -3, n_buffer : 10100, eps : 48.0


  0%|▎                                                                   | 152/40000 [00:25<2:08:33,  5.17it/s]

...

100%|█████████████████████████████████████████████████████████████████▊| 39902/40000 [1:41:38<00:10,  9.24it/s]

39900 에서 챙기는 아이템 : ['H', 'L', 'M']
마지막 행동 : ↑
 그 때의 마지막 상태 :
[[0.  0.  0.  0.6 0.  0.  0.  0.  0. ]
 [1.  1.  1.  1.  1.  0.3 1.  1.  1. ]
 [0.  1.  1.  1.  1.  1.  1.  1.  0. ]
 [0.  1.  0.  1.  0.  1.  0.  1.  0. ]
 [0.  1.  0.  1.  0.  1.  0.  1.  0. ]
 [0.  1.  0.  1.  0.  1.  0.  1.  0. ]
 [1.  1.  0.  1.  0.  1.  0.  1.  1. ]
 [1.  1.  1.  1.  1.  1.  1.  1.  1. ]
 [1.  1.  1.  1.  1.  1.  1.  1.  1. ]
 [0.  0.  0.  0.  1.  0.  0.  0.  0. ]]
마지막 보상 : -0.1
 총 받은 보상 :-9.99999999999998
score = -8, n_buffer : 50000, eps : 1.0


100%|█████████████████████████████████████████████████████████████████▉| 39952/40000 [1:41:44<00:05,  8.81it/s]

39950 에서 챙기는 아이템 : ['H', 'L', 'M']
마지막 행동 : ↑
 그 때의 마지막 상태 :
[[0.  0.  0.  0.3 0.  0.  0.  0.6 0. ]
 [1.  1.  1.  1.  1.  1.  1.  1.  1. ]
 [0.  1.  1.  1.  1.  1.  1.  1.  0. ]
 [0.  1.  0.  1.  0.  1.  0.  1.  0. ]
 [0.  1.  0.  1.  0.  1.  0.  1.  0. ]
 [0.  1.  0.  1.  0.  1.  0.  1.  0. ]
 [1.  1.  0.  1.  0.  1.  0.  1.  1. ]
 [1.  1.  1.  1.  1.  1.  1.  1.  1. ]
 [1.  1.  1.  1.  1.  1.  1.  1.  1. ]
 [0.  0.  0.  0.  1.  0.  0.  0.  0. ]]
마지막 보상 : -0.1
 총 받은 보상 :0.10000000000001436
score = -7, n_buffer : 50000, eps : 1.0


100%|██████████████████████████████████████████████████████████████████| 40000/40000 [1:41:49<00:00,  6.55it/s]
